In [1]:
import pandas as pd
from src.data import *
from src.portfolio import *
from src.value_at_risk import *
import yfinance as yf
from datetime import datetime as dt
from scipy.optimize import Bounds, LinearConstraint

In [2]:
esg_data = pd.read_excel('ESG_DATA1.xlsx')

for index, row in esg_data.iterrows():
    if row['CurrencyCode'] == 'SEK':
        esg_data.at[index, 'stock_symbol'] += '.ST'
    elif row['CurrencyCode'] == 'NOK':
        esg_data.at[index, 'stock_symbol'] += '.OL'
    elif row['CurrencyCode'] == 'DKK':
        esg_data.at[index, 'stock_symbol'] += '.CO'

In [3]:
# extract unique stock symbols
symbols = esg_data['stock_symbol'].unique()

# create a new dataframe to store the monthly closing data
monthly_close = pd.DataFrame()

# create a new dataframe to store the monthly closing data
monthly_close_esg = pd.DataFrame()

# create a new dataframe to store the monthly closing data
monthly_close_total = pd.DataFrame()

# loop over each stock symbol and retrieve monthly data using yfinance
for symbol in symbols:
    # retrieve data from yfinance
    stock_data = yf.download(symbol, start='2013-01-01', end='2023-01-01', interval='1mo', progress=False)
    
    # extract the 'Close' column and rename it with the stock symbol
    stock_data = stock_data[['Adj Close']].rename(columns={'Adj Close': symbol})
    monthly_close = pd.concat([monthly_close, stock_data], axis=1)

    # add the weighted score for the stock
    total_score = esg_data.loc[esg_data['stock_symbol']==symbol, 'total'].iloc[0]
    monthly_close_esg.index = monthly_close.index
    monthly_close_esg[symbol + '_ESG_total'] = total_score
    
    stock_data[symbol + '_ESG_total'] = total_score
    
    # append the stock data to the monthly_close_df
    monthly_close_total = pd.concat([monthly_close_total, stock_data], axis=1)

# print the result
#print(monthly_close_df)




1 Failed download:
- JOBS.ST: No data found for this date range, symbol may be delisted


/var/folders/7m/y8qq_4j92j1g6c6q2zclh3pw0000gn/T/ipykernel_79330/929332324.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  monthly_close_esg[symbol + '_ESG_total'] = total_score
/var/folders/7m/y8qq_4j92j1g6c6q2zclh3pw0000gn/T/ipykernel_79330/929332324.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  monthly_close_esg[symbol + '_ESG_total'] = total_score
/var/folders/7m/y8qq_4j92j1g6c6q2zclh3pw0000gn/T/ipykernel_79330/929332324.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of cal


1 Failed download:
- MSEIS.OL: No timezone found, symbol may be delisted


/var/folders/7m/y8qq_4j92j1g6c6q2zclh3pw0000gn/T/ipykernel_79330/929332324.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  monthly_close_esg[symbol + '_ESG_total'] = total_score
/var/folders/7m/y8qq_4j92j1g6c6q2zclh3pw0000gn/T/ipykernel_79330/929332324.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  monthly_close_esg[symbol + '_ESG_total'] = total_score
/var/folders/7m/y8qq_4j92j1g6c6q2zclh3pw0000gn/T/ipykernel_79330/929332324.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of cal

In [5]:
df_esg = monthly_close_esg.iloc[:, 15:20]
df = monthly_close.dropna(axis=1)
df = df.iloc[:,:5]
plot_cummulative_portfolio_returns(df)

In [ ]:
w = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
portfolio_return(df, w)

In [ ]:
portfolio_covariance(df)

In [ ]:
portfolio_mean(df)

In [ ]:
cov = portfolio_covariance(df)
portfolio_std(cov, w)

In [ ]:
portfolio_sharp_ratio(df, w, cov)

In [ ]:
#bounds = Bounds(-2, 5)

#Create x0, the first guess at the values of each asset's weight.
#w0 = np.linspace(start=1, stop=0, num=cov.shape[1])
#x0 = w0/np.sum(w0)
# All weights between 0 and 1
# # The second boundary is the sum of weights.
#linear_constraint = LinearConstraint(np.ones((cov.shape[1],), dtype=int),1,1)
#options = {'xtol': 1e-07, 'gtol': 1e-07, 'barrier_tol': 1e-07, 'maxiter': 1000}
 

In [ ]:
def portfolio_minimize_risk(port_return, 
                                port_covariance, 
                                esg_data, 
                                x0,
                                linear_constraint, 
                                bounds, 
                                options = None):
    """
    Function that will take different inputs including esg score data and compute the minimum risk of different portfolios 
    :param: A dataframe of the portfolio covariance matrix
    :param: A dataframe of esg scores of the different assets in portfolio
    :param: x0 argument that is the initial guess for the minimizer
    :param: Linear constraints for the minimizer
    :param: Bounds for the minimizer
    :param: 
    :param: Options for the minimizer
    :returns: A dataframe containing portfolio weight choice for minimizing portfolio risk using esg scores
    """
    
    results = {'esg':[],
               'weights':[],
               'risk':[],
               'return':[]}
    
    function = lambda weight: portfolio_std(port_cov=port_covariance, weights=weight)
    constraint_esg = {'type': 'eq', 'fun': lambda weight: np.dot(weight, esg_data)}
    result = minimize(function, 
                      x0, 
                      method='trust-constr', 
                      bounds=bounds, 
                      constraints=[linear_constraint, constraint_esg], 
                      options=options)
   
    optimal_weights = list(result['x'])
    print(optimal_weights)
    optimal_esg = np.dot(optimal_weights, esg_data)
    results['esg'].append(optimal_esg)
    results['weights'].append(optimal_weights)
    results['risk'].append(result['fun'])
    results['return'].append(np.dot(optimal_weights, port_return.sum()))

    return results


In [ ]:
#portfolio_minimize_risk(df, cov, df_esg, x0, linear_constraint, bounds)

In [ ]:
# set the window size
#window_size = 2

# make different lists to append data in every window
#expected_return = []
#expected_year = []

#def simple_return(beginning_price, end_price):
#  return ((end_price-beginning_price)/beginning_price)

#def annual_return(simple_return, years_held):
#  return (((simple_return+1)**(1/years_held))-1)

# setup af loop to iterate through window and make calculations
#for i in range(0, 6):

  # define the rolling window
#  sample_rolling_window = df[i*12:i*12+(12*window_size)]

  # calculate the expected return as a dataframe
#  window_monthly_returns = pd.DataFrame()
#  window_monthly_returns = sample_rolling_window
#  simple_window_yearly_returns_mean = simple_return(window_monthly_returns.iloc[0], window_monthly_returns.iloc[-1])
#  annual_window_return = annual_return(simple_window_yearly_returns_mean, window_size)

  # append the results of expected return and the years to list
#  expected_return.append(annual_window_return)

# make list of expected return into a dataframe
#for x in range(2013, 2024):
#  expected_year.append(dt(x,1,1))
  
#expected_return = pd.DataFrame(expected_return, index=expected_year)

#expected_return


In [ ]:
#esg_data = pd.read_excel('ESG_DATA.XLS')
#for index, row in esg_data.iterrows():
    # stockholm exchange
    if row['Isin'].startswith('SE'):
        esg_data.at[index, 'stock_symbol'] = row['stock_symbol'] + '.ST'
    
    # swiss exchange
    elif row['Isin'].startswith('CH0242214887'):
        esg_data.at[index, 'stock_symbol'] = row['stock_symbol'] + '.ST'
    elif row['Isin'].startswith('CH1122548808'):
        esg_data.at[index, 'stock_symbol'] = row['stock_symbol'] + '.OL'
    elif row['Isin'].startswith('CH'):
        esg_data.at[index, 'stock_symbol'] = row['stock_symbol'] + '.SW'
    
    # toroto exchange
    elif row['Isin'].startswith('CA98936C1068'):
        esg_data.at[index, 'stock_symbol'] = row['stock_symbol'] + '.L'
    elif row['Isin'].startswith('CA'):
        esg_data.at[index, 'stock_symbol'] = row['stock_symbol'] + '.TO'
    elif row['Isin'].startswith('GB'):
        esg_data.at[index, 'stock_symbol'] = row['stock_symbol'] + '.L'
    
    # paris exchange
    elif row['Isin'].startswith('FR'):
        esg_data.at[index, 'stock_symbol'] = row['stock_symbol'] + '.PA'

    # copenhagen exchange
    
    elif row['Isin'].startswith('DK0060668796'):
        esg_data.at[index, 'stock_symbol'] = row['stock_symbol'] + '.ST'
    elif row['Isin'].startswith('DK0060520450'):
        esg_data.at[index, 'stock_symbol'] = row['stock_symbol'] + '.OL'
    elif row['Isin'].startswith('FI4000297767'):
        esg_data.at[index, 'stock_symbol'] = row['stock_symbol'] + '.CO' 
    elif row['Isin'].startswith('DK'):
        esg_data.at[index, 'stock_symbol'] = row['stock_symbol'] + '.CO'

    # oslo exchange
    elif row['Isin'].startswith('NO'):
        esg_data.at[index, 'stock_symbol'] = row['stock_symbol'] + '.OL'
    elif row['Isin'].startswith('KYG'):
        esg_data.at[index, 'stock_symbol'] = row['stock_symbol'] + '.OL' 
    elif row['Isin'].startswith('CY'):
        esg_data.at[index, 'stock_symbol'] = row['stock_symbol'] + '.OL'
    elif row['Isin'].startswith('BMG'):
        esg_data.at[index, 'stock_symbol'] = row['stock_symbol'] + '.OL'
    elif row['Isin'].startswith('MHY'):
        esg_data.at[index, 'stock_symbol'] = row['stock_symbol'] + '.SG'
    elif row['Isin'].startswith('US36467X2062'):
        esg_data.at[index, 'stock_symbol'] = row['stock_symbol'] + '.OL'
    elif row['Isin'].startswith('SGX'):
        esg_data.at[index, 'stock_symbol'] = row['stock_symbol'] + '.OL'
    elif row['Isin'].startswith('FO0000000179'):
        esg_data.at[index, 'stock_symbol'] = row['stock_symbol'] + '.OL'

    elif row['Isin'].startswith('MT0000580101'):
        esg_data.at[index, 'stock_symbol'] = row['stock_symbol'] + '.DE'
    elif row['Isin'].startswith('MT0001390104'):
        esg_data.at[index, 'stock_symbol'] = row['stock_symbol'] + '.ST'
    elif row['Isin'].startswith('MT'):
        esg_data.at[index, 'stock_symbol'] = row['stock_symbol'] + '.SG'
   